In [ ]:
import random
from torch import nn
import time
import torch as T
import torch
# device = T.device("cuda")
device = T.device("cpu") 

In [ ]:
class FinalEmbedding:
    def __init__(self,x,y):
        self.x_data = x
        self.y_data = y
      
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        preds = self.x_data[idx]
        trgts = self.y_data[idx] 
        sample = { 
        'predictors' : preds,
        'targets' : trgts
        }
        return sample

In [ ]:
class pycoder_parameters:

    ''' Path '''
    Path = 'gen1_33'
    
    ''' Core Fuzzing Parameters '''
    NUM_FUZZ_PER_API= 100001 #2000
    FLOAT_TENSOR = False #We either generate float or integer tensors
    UNIT_TEST = False

    ''' Fuzzing Detailed Parameters '''
    MAX_TENSOR_DIMENSIONS = 3 #how many rows, columns, etc.
    MIN_VAL_PER_DIMENSION = 1 # e.g., min number of rows, columns, etc. 
    MAX_VAL_PER_DIMENSION = 5 # e.g., max number of rows, columns, etc. 

    #So far limiting to integers
    MIN_TENSOR_VALUE = 1
    MAX_TENSOR_VALUE = 15
    

    ''' Embedding Parameters '''
    EMBEDDING_NOISE_LEVEL = 0 #0 noise by default
    EMBEDDING_SIZE = 150
    SHAPE_EMBEDDING_SIZE = 6


    data_type = 'float' if FLOAT_TENSOR is  True else 'integer'
    file_name = 'Composite_' + str(NUM_FUZZ_PER_API) + '_' + data_type
    # file_name = '3seq_synthetic'
    fuzzing   = file_name + '.pt'
    embedding = file_name + '_training_embedding' + '.pt',
    classification = file_name + '.model_result' + '.pt' 

    def setNoiseLevel(self, noise):
        self.EMBEDDING_NOISE_LEVEL = noise
        # self.embedding = self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '.pt'
        self.embedding = self.file_name + '.embedding.pt'

    def getEmbeddingFile(self, noise):
        # return(self.file_name + '_training_embedding' + '_' + str(noise) + '.pt')
        return(self.file_name + '_training_embedding.pt')

    def getVisulizationFile(self, noise):
        # return(self.file_name + '_training_embedding' + '_' + str(noise) + '_' +  'tSNE.pt')
        return(self.file_name + '_training_embedding_tSNE.pt')
noise = 0
f = pycoder_parameters()
f.embedding = f.getEmbeddingFile(noise)
print(f.embedding)
print(f.getVisulizationFile(noise))

In [ ]:
api2indx = torch.load(f.Path + '/api2indx.pt')
api2indx

In [ ]:
def process_dataX(io_seq_tensors):
    
    io_seq = []

    for tensor_list in io_seq_tensors:
        
        n0 = tensor_list[0]

        if(len(tensor_list) == 1):
            n1 = torch.zeros(n0.shape)
            n2 = torch.zeros(n0.shape)
            
        elif(len(tensor_list) == 2):
            n1 = tensor_list[1]
            n2 = torch.zeros(n0.shape)

        elif(len(tensor_list) == 3):
            n1 = tensor_list[1]
            n2 = tensor_list[2]
            
        new_list = torch.stack((n0, n1, n2))
        
        io_seq.append(new_list)
        
    return(torch.stack(io_seq))


In [ ]:
indx2api = {}
EOS = '<eol>'

def process_dataY(api_seq_list):
    global indx2api
    global api2indx
    ''' Add <eol> to the dictionary '''
    indx2api = {v: k for k, v in api2indx.items()}
    if api2indx.get(EOS, -1) == -1:
        max_key = max(indx2api.keys())
        print(max_key)
        indx2api[max_key+1] = EOS
        api2indx[EOS] = max_key+1
    eos = api2indx[EOS]
    api_tensors = []

    for api_seq in api_seq_list:
        api0 = api_seq[0]

        if len(api_seq) == 1:
            api1 = eos
            api2 = eos

        elif len(api_seq) == 2:
            api1 = api_seq[1]
            api2 = eos

        elif len(api_seq) == 3:
            api1 = api_seq[1]
            api2 = api_seq[2]

        else:
            print('!!! Not supposed to be here')

        t = torch.tensor([api0, api1, api2])
        api_tensors.append(t)
    
    return(torch.stack(api_tensors))

In [ ]:
def shuffle_dataset(X_dataset, Y_dataset):

    print('len orig_dataset', len(X_dataset),  len(Y_dataset))
  
    length = len(X_dataset)

    idx = list(range(length))  # indices to all elements
    random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
    data_idx = idx[:]

    x = []
    y = []

    x = [X_dataset[i] for i in data_idx]
    y = [Y_dataset[i] for i in data_idx]

    print(len(x), len(y))
    return(x,y)

''' Test '''
x = [1,2,3,4]
y = [5,6,7,8]

x, y = shuffle_dataset(x,y)
print(x)
print(y)

In [ ]:
i = 9
embed_file = f.Path + '/' +str(i*10000) + '_test_embedding.pt' 
print(embed_file)
X_test, y_test = torch.load(embed_file)

In [ ]:
X_test1=X_test.copy()
X_test = X_test1[:10000]

In [ ]:
y_test1=y_test.copy()
y_test = y_test1[:10000]

In [ ]:
test_set_x = process_dataX(X_test)
test_set_y = process_dataY(y_test)

In [ ]:
torch.save((test_set_x,test_set_y), f.Path + '/' + 'test_10000.pt')

In [ ]:
test_set_x,test_set_y = torch.load(f.Path + '/' + 'test_10000.pt')

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=True)   
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim*2, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out1 = out.contiguous().view(-1, self.hidden_dim*2)
        out1 = self.fc(out1)
        
        return out1, hidden, out
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [ ]:
class FFNet(T.nn.Module):
  def __init__(self):
    super(FFNet, self).__init__()

    self.hid1 = T.nn.Linear(4*(f.EMBEDDING_SIZE+f.SHAPE_EMBEDDING_SIZE+1+2), 500)
    self.hid2 = T.nn.Linear(500, 250)
    self.hid3 = T.nn.Linear(250, 100)
    self.oupt = T.nn.Linear(100, len(api2indx))

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

    T.nn.Dropout(p=0.2)


  def forward(self, x):
    z1 = T.tanh(self.hid1(x))
    z2 = T.tanh(self.hid2(z1))
    z3 = T.tanh(self.hid3(z2))
    z = self.oupt(z3)  # no softmax: CrossEntropyLoss() 
    return (z, z3, z2, z1)


In [ ]:
def api_edit_distance(seq1, seq2):

    edit_distantce = 0

    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            edit_distantce += 1

    return(edit_distantce)

In [ ]:
def beam_search(top3_list):

    api_seq = []
    api_seq1 = []
    for i in top3_list[0]:
        for j in top3_list[1]:
            api_seq1.append(((i[0], j[0]), i[1]*j[1]))
    
    api_seq1.sort(key = lambda x: x[1], reverse=True) 

    for k in top3_list[2]:
        for s1 in api_seq1:
            api_seq.append(((s1[0][0], s1[0][1], k[0]), s1[1]*k[1]))

    api_seq.sort(key = lambda x: x[1], reverse=True) 
    
    return(api_seq)

In [ ]:
top_result = {}
seq_result = {}
seq_total = {}
mrp = 0

def query_model(X, Y):
    global mrp
    with T.no_grad():
      start_time = time.time()
      predicts, z3, z2, z1 = net(X)
      temp_z3 = torch.unsqueeze(z3,0)
      model_output, hidden, int_output = rnn_model(temp_z3)

      target_list = list(Y.cpu().numpy())
    
    top_indx = []

    top3_list = []

    if DEBUG:
        print('---- Top-k ----')
    for i, m in enumerate(model_output):
        top3 = []
        prob = nn.functional.softmax(m, dim=0).data
        # Taking the class with the highest probability score from the output
        xx = torch.topk(prob,3,dim=0)[1]
        for x in xx.cpu().numpy():
          if DEBUG:
            print(indx2api[x])
          top3.append((indx2api[x], prob[x].item()))
        top3_list.append(top3)
        if DEBUG:
            print('====')
    
    if DEBUG:
        print('---- Predicted ----')
    for m in model_output:
        prob = nn.functional.softmax(m, dim=0).data
        api_ind = torch.max(prob, dim=0)[1].item()
        top_indx.append(api_ind)
        if DEBUG:
            print(indx2api[api_ind], prob[api_ind])

    if DEBUG:
        print('---- Expected ----')
        for o in list(Y):
            print(indx2api[o.item()])

    o = list(Y)
    expected = ((indx2api[o[0].item()], indx2api[o[1].item()], indx2api[o[2].item()]))

    if seq_result.get(expected, -1) == -1:
        seq_result[expected] = 0
        seq_total[expected] = 0

    seq_total[expected] += 1
    
    top3_list = beam_search(top3_list)
    for i,t in enumerate(top3_list):
      if t[0] == expected:
        if top_result.get(i+1, -1) == -1:
            top_result[i+1] = 0
        top_result[i+1] += 1

        mrp += 1/(i+1)
        
        if i < 6:
          seq_result[expected] += 1
        return 1

    return(0)

In [ ]:
DEBUG = False

def validation_top3(model, ds):
  edit_distance = {0:0,1:0,2:0,3:0}
  n_correct = 0
  total = len(ds)
  acc_seq_length = {}
  eol_indx = api2indx['<eol>']
  Decision = True
  seq_length_dict = {1:0,2:0,3:0}

  n_correct = 0; n_wrong = 0
  for i in range(len(ds)):
    X = ds[i]['predictors'].to(device)
    Y = ds[i]['targets'].to(device)  # [0] [1] or [2]

    n_correct += query_model(X,Y)
    
  return(n_correct/total)

In [ ]:
max_epochs = 20

In [ ]:
net = torch.load(f.Path + '/' + str(max_epochs) + '_train_net_model.pt')
rnn_model = torch.load(f.Path + '/' + str(max_epochs) + '_train_rnn_model.pt')

In [ ]:
top_result = {}
seq_result = {}
seq_total = {}
s2p = {}
total_length = 0
# X_test, y_test = torch.load(f.Path + '/training_embeddings/3seq_synthetic_test_embedding.pt' )
X_test, y_test = torch.load(f.Path + '/test_10000.pt')
test_set_x = process_dataX(X_test)
test_set_y = process_dataY(y_test)
test_ds = FinalEmbedding(test_set_x,test_set_y)
validation_top3(rnn_model, test_ds) 
count = 0
for i in sorted (top_result) :
    count += top_result[i]
    print (i, top_result[i],count*100/len(test_set_y), end ="\n")
print('-----------------------------------------------------------')
for i in seq_result:
    s2p[i] = seq_result[i]*100/seq_total[i]
    print("%r, %0.2f " % (i, seq_result[i]*100/seq_total[i]))


In [ ]:
torch.save(s2p, f.Path +'/s2p.pt')

In [ ]:
torch.save(test_ds, f.Path +'/test_ds.pt')

In [ ]:
DEBUG = False

def save_for_analysis(model, ds):
  edit_distance = {0:0,1:0,2:0,3:0}
  n_correct = 0
  total = len(ds)
  acc_seq_length = {}
  eol_indx = api2indx['<eol>']
  print('eol index', eol_indx)
  Decision = True
  seq_length_dict = {1:0,2:0,3:0}

  hidden_states = []

  n_correct = 0; n_wrong = 0
  for i in range(len(ds)):
    X = ds[i]['predictors'].to(device)
    #print(X)
    Y = ds[i]['targets'].to(device)  # [0] [1] or [2]

    with T.no_grad():
      start_time = time.time()
      predicts, z3, z2, z1 = net(X)
      #print(z3.shape)
      temp_z3 = torch.unsqueeze(z3,0)
      #print(temp_z3.shape)
      model_output, hidden, int_output = model(temp_z3)

      target_list = list(Y.cpu().numpy())
      
      if DEBUG:
        for o in list(Y):
          print(indx2api[o.item()])
        print('-----')

      top_indx = []

      for m in model_output:
        prob = nn.functional.softmax(m, dim=0).data
        # Taking the class with the highest probability score from the output
        api_ind = torch.max(prob, dim=0)[1].item()
        top_indx.append(api_ind)
        if DEBUG:
          print(indx2api[api_ind])
      
      ed = 0
      hidden_states.append((X,int_output,target_list))
      
      
      if target_list == top_indx:
        n_correct+=1
        edit_distance[0] += 1
        Decision = True
      else:
        ed = api_edit_distance(target_list,top_indx)
        edit_distance[ed] += 1
        Decision = False

      seq_length = 3 - target_list.count(eol_indx)
      seq_length_dict[seq_length] += 1
      if DEBUG:
        print('seq_length', seq_length)
      
      if (acc_seq_length.get((seq_length,Decision), -1)) == -1:
        acc_seq_length[(seq_length,Decision)] = 0
      acc_seq_length[(seq_length,Decision)] += 1

      if DEBUG:
        print('======', ed, '======')

  return(hidden_states)
  

In [ ]:
first_layer = []
second_layer = []
third_layer = []

h = save_for_analysis(rnn_model, test_ds) 

In [ ]:
for ih in h:
    
    x, int_val, y  = ih[:]
    first_layer.append((x[0], int_val[0][0], y[0]))
    second_layer.append((x[1], int_val[0][1], y[1]))
    third_layer.append((x[2], int_val[0][2], y[2]))